# Worksheet 08

Name: Tristan Lee
UID: U24272030

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [40]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.99028640788274, 5.983865556888575, 4.497879431029041, 3.6541646183444403, 4.6284893897784825, 6.140305301078399, 5.2351246284288635, 6.675454034798956, 4.787315074869976, 4.732381582389137]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [41]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[10.060850384628726, 8.819206045232145, 7.208630741485397, 5.679303656681893, 6.717370664043483, 7.515861175829191, 7.023159985821022, 8.735684019022884, 9.057116734578575, 6.741573126937684]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [42]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p1 = s2.pop()
        data.append(p1)
print(data)

[6.741573126937684, 4.732381582389137, 4.787315074869976, 6.675454034798956, 9.057116734578575, 5.2351246284288635, 6.140305301078399, 4.6284893897784825, 3.6541646183444403, 4.497879431029041]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

We need the priors for both s1 and s2, and the mean and variances for the distributions of their weights (so 6 total parameters). We know the means and variances for each species since we initializes them, but not the priors.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [43]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.732381582389137, 4.787315074869976, 5.2351246284288635, 4.6284893897784825, 3.6541646183444403, 4.497879431029041]
[6.741573126937684, 6.675454034798956, 9.057116734578575, 6.140305301078399]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.589225787473324,  mean_2 = 7.153612299348404
var_1 = 0.2268572395768833,  var_2 = 1.262132953310437


/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The values are in the right neighborhood; the P(S_i) are not exactly 0.5 but are pretty close. The means are also pretty close. The variances are a bit off, especially var_1, which seems to be a result of the random points being really close to the guessed mean.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [44]:
from scipy.stats import norm
from math import sqrt

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.741573126937684
probability of observing that point if it came from cluster 0 =  3.081202944307527e-05
probability of observing that point if it came from cluster 1 =  0.3320074825585267
point =  4.732381582389137
probability of observing that point if it came from cluster 0 =  0.8006032224847547
probability of observing that point if it came from cluster 1 =  0.03481380436471897
point =  4.787315074869976
probability of observing that point if it came from cluster 0 =  0.7681993074077443
probability of observing that point if it came from cluster 1 =  0.03863663658961974
point =  6.675454034798956
probability of observing that point if it came from cluster 0 =  5.7144667815551306e-05
probability of observing that point if it came from cluster 1 =  0.32435552514963395
point =  9.057116734578575
probability of observing that point if it came from cluster 0 =  6.537394856432445e-20
probability of observing that point if it came from cluster 1 =  0.0845223538149004
point =  5.2

For each point, the predicted cluster is the one with the highest probability. Using this, the predictions are all nearly correct except for the point 6.675..., which actually came from S_1.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [46]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.576954123292188,  P(S_2) = 0.4230458767078119
mean_1 = 4.581185615259196,  mean_2 = 7.024879866599645
var_1 = 0.23405821609868094,  var_2 = 1.481254771166775


It looks like the estimates are further from what they should be. This is probably just because the random initialization of the species weights wasn't representative of the actual parameters.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [47]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_c[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  6.741573126937684
probability of observing that point if it came from cluster 0 =  3.856469008066481e-05
probability of observing that point if it came from cluster 1 =  0.3190280921621556
point =  4.732381582389137
probability of observing that point if it came from cluster 0 =  0.7853071963082268
probability of observing that point if it came from cluster 1 =  0.05560943693091169
point =  4.787315074869976
probability of observing that point if it came from cluster 0 =  0.7530584239370447
probability of observing that point if it came from cluster 1 =  0.06048245863760167
point =  6.675454034798956
probability of observing that point if it came from cluster 0 =  7.03361470139908e-05
probability of observing that point if it came from cluster 1 =  0.3145545311979511
point =  9.057116734578575
probability of observing that point if it came from cluster 0 =  2.136807152946683e-19
probability of observing that point if it came from cluster 1 =  0.08131155459151175
point =  5.235

The assignments that these new probabilities would lead to are the same as the previous iteration, but most of the assignments are more "unsure" of this; that is, the maximum probability for most of the points decreased from the previous iteration to this one.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [52]:
assignments = zip(data, map(lambda x: 0 if x[0]>x[1] else 1, zip(prob_s0_x, prob_s1_x)))
for assignment in assignments:
    print("Point {}, Cluster {}".format(*assignment))

Point 6.741573126937684, Cluster 1
Point 4.732381582389137, Cluster 0
Point 4.787315074869976, Cluster 0
Point 6.675454034798956, Cluster 1
Point 9.057116734578575, Cluster 1
Point 5.2351246284288635, Cluster 0
Point 6.140305301078399, Cluster 1
Point 4.6284893897784825, Cluster 0
Point 3.6541646183444403, Cluster 0
Point 4.497879431029041, Cluster 0
